In [32]:
from tqdm import tqdm
import pandas as pd
import datetime as dt
import numpy as np
import warnings
import pymssql
import locale
import glob
import os

warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)
pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [3]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Название склада], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Название товара], [Остаток], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [ШК], [Код товара]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [4]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [5]:
stock = stock[stock['ID Сайта'] != '']

In [6]:
stock['Код склада'] = stock['Код склада'].astype(int)

In [28]:
year_now, month_now = str(dt.datetime.today().year), str(dt.datetime.today().month)
if int(month_now) < 10: month_now = '0' + str(month_now)

In [29]:
os.chdir(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new')
extension = 'xlsx'
filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames = list()

for file in filenames:
    if year_now in file and month_now in file: 
        all_filenames.append(file) 

In [31]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

In [34]:
sale.columns = ['КБК', 'Дата', 'Продавец', 'Номер чека', 'ID продажи', 'Тов',
    'article', 'Количество', 'Номенклатура', 'Номер карты',
    'Товарная группа', 'Товарное направление', 'Тип 1',
    'Списание СберСпасибо', 'Группа наценки', 'Вес', 'Проба', 'Размер',
    'ШК', 'Цена без скидки', 'Базовая скидка', 'Скидка в руб.', 'Название',
    'Цена после скидки', 'Phone', 'Себестоимость', 'Сумма', 'Услуга',
    'Общая сумма']

In [35]:
articles = stock['Артикул поставщика'].tolist()

In [36]:
sale = sale.query('article in @articles')

In [37]:
sale = sale.groupby(by = 'article', as_index=False)[['Количество', 'Общая сумма']].sum()

In [38]:
sale.columns = ['Артикул поставщика', 'Количество продаж за предыдущую неделю', 'Общая сумма продаж за предыдущю неделю']

In [39]:
center = stock[stock['Код склада'] == 3075]
center = center.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
center.columns = ['ID Сайта', 'Остаток на центральном складе']

In [40]:
kolambus = stock[stock['Код склада'] == 5442]
kolambus = kolambus.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
kolambus.columns = ['ID Сайта', 'Остаток на складе коламбуса']

In [42]:
federal = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Федеральные склады 11.06.xlsx')

In [43]:
stock.rename(columns={'Название склада': 'name'}, inplace=True)

In [44]:
federal = federal['outlet_name'].tolist() 

In [45]:
federal = stock.query('name in @federal')

In [46]:
federal = federal.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
federal.columns = ['ID Сайта', 'Остаток на федеральных складах']

In [47]:
stock = stock.merge(federal, how='outer', on='ID Сайта')
stock = stock.merge(kolambus, how='outer', on='ID Сайта')
stock = stock.merge(center, how='outer', on='ID Сайта')

In [48]:
conn = pymssql.connect(server='spb-s-sql-dwh', database='DWH_DEV', charset='WINDOWS-1251')
cursor = conn.cursor()

sql = f'''
SELECT [article], [sort]
from [DWH_DEV].[ecom].[catalog_product]
'''

sql_query = pd.read_sql_query(sql, conn)
sort = pd.DataFrame(sql_query)

In [49]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5758254 entries, 0 to 5758253
Data columns (total 14 columns):
 #   Column                                           Dtype  
---  ------                                           -----  
 0   Код склада                                       int32  
 1   name                                             object 
 2   Артикул поставщика                               object 
 3   Внутренний артикул                               object 
 4   ID Сайта                                         object 
 5   Название товара                                  object 
 6   Остаток                                          float64
 7   Цена Закупки за 1 шт. (упр сс с НДС за единицу)  float64
 8   ШК                                               object 
 9   Код товара                                       object 
 10  Код объекта                                      object 
 11  Остаток на федеральных складах                   float64
 12  Остаток на скл

In [50]:
sort.rename(columns={'article': 'ID Сайта'}, inplace=True)

In [51]:
stock = stock.merge(sort, how='outer', on='ID Сайта')

In [52]:
stock = stock.merge(sale, how='outer', on='Артикул поставщика')

In [53]:
summ = stock.groupby(by='ID Сайта', as_index=False)['Остаток'].sum()

In [54]:
summ.rename(columns={'Остаток': 'Общий остаток на всех складах'}, inplace=True)

In [55]:
stock = stock.merge(summ, how='left', on='ID Сайта')
stock = stock.drop_duplicates(subset=['ID Сайта', 'Общий остаток на всех складах'])

In [56]:
stock['Остаток на складе коламбуса'].fillna(0, inplace=True)
stock['Остаток на федеральных складах'].fillna(0, inplace=True)
stock['Остаток на центральном складе'].fillna(0, inplace=True)

In [57]:
stock['Сумма в себестоимости на остатках ОБЩ'] = stock['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] * stock['Общий остаток на всех складах']
stock['Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР'] = (stock['Остаток на складе коламбуса'] + stock['Остаток на федеральных складах'] + stock['Остаток на центральном складе']) * stock['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [58]:
stock

,Код склада,name,Артикул поставщика,Внутренний артикул,ID Сайта,Название товара,Остаток,Цена Закупки за 1 шт. (упр сс с НДС за единицу),ШК,Код товара,Код объекта,Остаток на федеральных складах,Остаток на складе коламбуса,Остаток на центральном складе,sort,Количество продаж за предыдущую неделю,Общая сумма продаж за предыдущю неделю,Общий остаток на всех складах,Сумма в себестоимости на остатках ОБЩ,Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР
0,5112.00,"Краснодар, Нефтяников, 42, ТЦ «Сказка»(Дубль)",SL-R01641-X-GB-X-XBR,SL-R01641-X-GB-X-XBR_СИ,1749451,"Кольцо с коричн.фиан. поз., чёрн.род.",1.00,941.00,2078600686279,ТОВ01444848,6-04-076,2.00,4.00,0.00,0,NaN,NaN,172.00,161852.00,5646.00
172,5112.00,"Краснодар, Нефтяников, 42, ТЦ «Сказка»(Дубль)",SL-R01641-X-WB-X-XBR,SL-R01641-X-WB-X-XBR_СИ,1947198,"Кольцо с коричн.фиан. бел., чёрн.род.",1.00,932.00,2078600679645,ТОВ01444874,6-04-076,0.00,1.00,0.00,0,2.00,2240.00,137.00,127684.00,932.00
309,5112.00,"Краснодар, Нефтяников, 42, ТЦ «Сказка»(Дубль)",SL-R01642-X-GB-BRXBR,SL-R01642-X-GB-BRXBR_СИ,1695943,"Кольцо с коричн.фиан. поз., чёрн.род.",1.00,993.00,2078600680176,ТОВ01444876,6-04-076,1.00,2.00,0.00,0,NaN,NaN,168.00,166824.00,2979.00
477,5112.00,"Краснодар, Нефтяников, 42, ТЦ «Сказка»(Дубль)",SL-R01642-X-WB-BRXBR,SL-R01642-X-WB-BRXBR_СИ,1395098,"Кольцо с коричн.фиан. бел., чёрн.род.",1.00,1005.00,2078600679467,ТОВ01444888,6-04-076,0.00,3.00,0.00,0,1.00,1005.00,183.00,183915.00,3015.00
660,5112.00,"Краснодар, Нефтяников, 42, ТЦ «Сказка»(Дубль)",CZ-E01195-X-GB-X-XBR,CZ-E01195-X-GB-X-XBR_СИ,1529463,"Серьги с коричн.фиан. поз., чёрн.род.",1.00,1802.00,2078600690426,ТОВ01444900,6-04-076,1.00,1.00,0.00,0,4.00,15600.00,123.00,221646.00,3604.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860673,NaN,NaN,NaN,NaN,2235328,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN
5860674,NaN,NaN,NaN,NaN,7741803,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN
5860675,NaN,NaN,NaN,NaN,2212903,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN
5860676,NaN,NaN,NaN,NaN,1357200,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN


In [60]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [61]:
images = images[['article', 'Ссылка на изображение', 'published']]

In [62]:
images.rename(columns={'article': 'ID Сайта'}, inplace=True)

In [63]:
images['ID Сайта'] = images['ID Сайта'].astype(str)

In [64]:
stock = stock.merge(images, how='outer', on='ID Сайта')

In [65]:
stock = stock.drop(columns=['Остаток', 'Код склада'])

In [67]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.07.12.xlsx')

In [68]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [69]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1','discount_value_promotions_2', 'discount_value_promotions_3','discount_value_promotions_4']
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [70]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1
    
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [71]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'Цена со скидками Ecom']]

In [72]:
ecom.columns = ['ID Сайта', 'Цена со скидками Ecom']
stock = stock.merge(ecom, how='left', on='ID Сайта')

In [73]:
stock = stock.drop_duplicates(subset=['ID Сайта'])

In [74]:
stock = stock.drop(columns=['name', 'Код объекта', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)'])

In [75]:
stock.columns =['Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Название товара', 'ШК', 'Код товара', 'Остаток на федеральных складах',
    'Остаток на складе коламбуса', 'Остаток на центральном складе', 'Ранг на сайте',
    'Количество продаж', 'Общая сумма продаж за июнь',
    'Общий остаток на всех складах',
    'Сумма в себестоимости на остатках ОБЩ',
    'Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР',
    'Ссылка на изображение', 'Опубликован', 'Цена со скидками Ecom']

In [77]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')['Артикул сайта'].astype(str).tolist()

In [78]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197330 entries, 0 to 400296
Data columns (total 18 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   Артикул поставщика                                    94906 non-null   object 
 1   Внутренний артикул                                    94906 non-null   object 
 2   ID Сайта                                              197330 non-null  object 
 3   Название товара                                       94906 non-null   object 
 4   ШК                                                    94906 non-null   object 
 5   Код товара                                            94906 non-null   object 
 6   Остаток на федеральных складах                        197330 non-null  float64
 7   Остаток на складе коламбуса                           197330 non-null  float64
 8   Остаток на центральном складе                    

In [79]:
stock['Запрет к размещению'] = np.nan
for i in stock.index:
    if stock['ID Сайта'][i] in zapret:
        stock['Запрет к размещению'][i] = True
    else:
        stock['Запрет к размещению'][i] = False

In [81]:
stock['Запрет к размещению'].value_counts()

Запрет к размещению
False    197286
True         44
Name: count, dtype: int64

In [82]:
stock.dropna(subset=['ШК'], inplace=True)

In [83]:
stock['Опубликован'].value_counts()

Опубликован
True     71329
False     4390
Name: count, dtype: int64

In [84]:
stock1 = stock[~stock['Ссылка на изображение'].astype(str).str.contains('http')]
# df = df[df['Ссылка на изображение'].astype(str).str.contains('http')]

In [85]:
stock2 = stock[(stock['Ссылка на изображение'].astype(str).str.contains('http')) & (stock['Запрет к размещению'] == False) & (stock['Опубликован'] == False)]

In [86]:
stock = pd.concat([stock1, stock2], ignore_index=True)

In [71]:
stock.to_excel('08.07.24 для контента.xlsx')